In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import random
## convert libsvm to sparse matrix
from scipy.sparse import csr_matrix
import numpy as np
import statistics
import glob

def read_libsvm(fname, num_features=0):
    '''
        Reads a libsvm formatted data and outputs the training set (sparse matrix)[1],
        the label set and the number of features. The number of features
        can either be provided as a parameter or inferred from the data.

        Example usage:
		X_train, y_train, num_features = read_libsvm('data_train')
		X_test, y_test, _ = read_libsvm('data_test', num_features)

		[1] https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
	'''
    data = []
    y = []
    row_ind = []
    col_ind = []
    with open(fname) as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            elements = line.split()
            y.append(int(elements[0]))
            for el in elements[1:]:
                row_ind.append(i)
                c, v = el.split(":")
                col_ind.append(int(c))
                data.append(float(v))
    if num_features == 0:
        num_features = max(col_ind) + 1
    X = csr_matrix((data, (row_ind, col_ind)), shape=(len(y), num_features+1))

    return X, np.array(y), num_features

def svm(X_df, y_t , epochs=200, rate=10,C=10):
    rgen = np.random.RandomState(0)
    #b = rgen.uniform(-0.01,0.01,1) #initialize bias
    s=X_df.shape[1]

    weights = rgen.normal(loc=0.0, scale=0.0, size=s)
    #w = np.zeros(len(X_df[0]))
    transpose =[]
    arr_acc=[]
    error=[]
    X_array = X_df.values
    #dot_product = np.dot(X_array,w)
    gamma_t=rate
    x=0
    t=0
    train=0
    for e in range(epochs):

        errors=0
        X_df.sample(frac=1)
        for i, row in X_df.iterrows():
            t=i+1
            gamma_t=float(rate)/(1+((rate*t)/C))
            #dot_product=map(lambda x,y:x*y,weights,X_array[i])
            #print((dot_product))
            #derived_label=reduce(lambda x,y:x+y,dot_product)
            dot_product = np.dot(weights,X_array[i])
            derived_label= np.sum(dot_product)
            actual_label=y_t[i]
            #print(derived_label,actual_label)
            if derived_label*actual_label<=1:
                train+=1
                update = X_array[i]*C*gamma_t*actual_label
                weights = weights * (1-gamma_t)
                weights = np.add(weights,update)
            else:
                gamma_t=1-gamma_t
                weights = weights*gamma_t

            '''
            condition = y_t[index]*dot_product[index]
            #print(condition)
            if(condition<= 0):
                w = w - w*lr + lr*C*y_t[index]* X_array[index]
                errors = errors+1
            else:
                w = (1-lr)*w
            '''

    return weights,errors


In [ ]:
def convert(col):
    #print(col)
    median = col.median()
    #print("m",median)
    col[col<median]=0
    col[col>median]=1
    return col



def read_data():
    X_train, y_train, num_features = read_libsvm("/kaggle/input/ml-fall2019-android-malware/data/data/data-splits/data.train",0)
    X_test, y_test, _ = read_libsvm("/kaggle/input/ml-fall2019-android-malware/data/data/data-splits/data.test",0)
    X_train_df = pd.DataFrame(X_train.toarray())
    X_test_df= pd.DataFrame(X_test.toarray())
    bias_array = np.full(X_train_df.shape[0],-1)
    
    x = X_train_df.values #returns a numpy array   
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    X_train_df = pd.DataFrame(x_scaled)
    
    #X_train_df = X_train_df.apply(convert,axis=0)
    
    
    X_train_df['bias']=bias_array
    y_train = np.where(y_train==1,1,1)
    
    x = X_test_df.values #returns a numpy array   
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    X_test_df = pd.DataFrame(x_scaled)
    
    #X_test_df = X_test_df.apply(convert,axis=0)
    
    y_test= np.where(y_test==1,1,-1)
    
    print(X_train_df.head())
    print(y_train)
    return X_train_df,y_train,X_test_df,y_test


In [ ]:

def accuracy(X_test_df,y_test,w_new):
    bias_array = np.full(X_test_df.shape[0],-1)
    X_test_df['bias']=bias_array
    X_test_array = X_test_df.values

    dot_product = np.dot(X_test_array,w_new)
    dot_product = (np.where(dot_product>=0,1,-1))
    x=np.count_nonzero(dot_product==y_test)
    y=len(y_test)
    acc = float(x/y)
    print("accuracy",x/y)

    return acc

In [ ]:

def main():

    X_train_df,y_train,X_test_df,y_test= read_data()
    #change 0 to -1
    y_train = np.where(y_train==1,1,-1)
    ''' 
    lr =[0.1,0.01,0.001,0.0001,10]
    C =[0.1,0.01,0.001,0.0001,10]
    for lr_2 in lr:
        for c in C:
            print(lr_2,"*******",c)
            w_new,errors = svm(X_train_df, y_train ,20,lr_2, c)
            #bias_array = np.full(X_train_df.shape[0],1)
            #X_train_df['bias']=bias_array
            X_train_array = X_train_df.values
            dot_product = np.dot(X_train_array,w_new)
            dot_product = (np.where(dot_product>=0,1,1))
            x=np.count_nonzero(dot_product==y_train)
            y=len(y_train)
            print("Train accuracy",x/y)
            print("******")
    '''
    acc_list=[]
    lr= [0.1,0.01,0.001,0.0001,10]
    C = [0.1,0.01,0.001,0.0001,10]
    for lr_2 in lr:
        for c in C:
            print(lr_2,"*******",c)
            w_new,errors = svm(X_train_df, y_train ,20,lr_2, c)
            bias_array = np.full(X_test_df.shape[0],-1)
            X_test_df['bias']=bias_array
            X_test_array = X_test_df.values
            dot_product = np.dot(X_test_array,w_new)
            dot_product = (np.where(dot_product>=0,1,-1))
            x=np.count_nonzero(dot_product==y_test)
            y=len(y_test)
            print("Test accuracy",x/y)
            print("******")
            acc_list.append(x/y)


main()


EVAL

In [ ]:
X_eval, y_eval, _ = read_libsvm('/kaggle/input/ml-fall2019-android-malware/data/data/data-splits/data.eval.anon', 0)
X_eval_df = pd.DataFrame(X_eval.toarray())    
print(X_eval_df.head())
x = X_eval_df.values #returns a numpy array   
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X_eval_df = pd.DataFrame(x_scaled)
X_eval_df = X_eval_df.apply(convert,axis=0)
print(X_eval_df.shape)

In [ ]:
X_train_df,y_train,X_test_df,y_test= read_data()
#change 0 to -1
print(X_train_df.shape)
y_train = np.where(y_train==1,1,-1)
w_new,errors = svm(X_train_df, y_train ,20,0.0001, 10)
bias_array = np.full(X_eval_df.shape[0],-1)
print(X_eval_df.shape)
X_eval_df['bias']=bias_array
X_eval_df_array = X_eval_df.values
dot_product = np.dot(X_eval_df_array,w_new)
dot_product = (np.where(dot_product>=0,1,-1))
len(dot_product)

In [ ]:
print(X_eval_df.shape)
print(X_eval_df.head())
print(X_train_df.shape)
print(X_train_df.head())

In [ ]:
eval_id_list=[]
with open("/kaggle/input/ml-fall2019-android-malware/data/data/data-splits/eval.id") as f:
    lines = f.readlines()
    
for ele in lines:
    ele=ele.rstrip("\n")
    print(ele)
    eval_id_list.append(ele)  

In [ ]:
dot_product= np.where(dot_product==-1,0,1)
len(dot_product)

In [ ]:
list_of_tuples = list(zip(eval_id_list, dot_product))  
    
# Assign data to tuples.  
list_of_tuples   
  
  
# Converting lists of tuples into  
# pandas Dataframe.  
df_new = pd.DataFrame(list_of_tuples, columns = ['example_id', 'label'])  
     
# Print data.  
df_new

In [ ]:
import collections, numpy

collections.Counter(dot_product)



In [ ]:
df_new.to_csv(r'df_new.csv')
from IPython.display import FileLink
FileLink(r'df_new.csv')